In [2]:
import pandas as pd
import warnings
from openpyxl import load_workbook

# Check the current working directory
import os
print(os.getcwd())

# Import the necessary module
import importlib
from helper.bom_template import write_df_to_excel

# If you've made changes to the bom_template.py file, reload it
importlib.reload(helper.bom_template)

# Suppress specific UserWarning from Pandas
warnings.filterwarnings("ignore", category=UserWarning, message="Data Validation extension is not supported")

# Read the BOM sheet with header in row 2
file_path = "../data/bom275.xlsx"
sheet_name = "BOM"
df_bom = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Drop rows where all cells are NaN
df_bom = df_bom.dropna(how="all")
# Drop columns where all values are NaN
df_bom = df_bom.dropna(axis=1, how="all")

# Display the total number of rows and columns
# print("Total rows and columns:", df_bom.shape)

# List all column names
column_names = df_bom.columns.tolist()
# print(column_names)

# Specify the columns you want to keep
columns_to_keep = ['所属装配体', '布局号', 'PLM编号', '层级深度', '父层级', '整套总数', 'ERP编号', '物料描述', '单层数量', '单位', '物料种类']

# Select only these columns
df_bom = df_bom[columns_to_keep]

# Convert all values in 'PLM编号' to string before filtering
df_bom['PLM编号'] = df_bom['PLM编号'].astype(str)

# Drop rows where 'PLM编号' is NaN or not a string (we know all are now strings)
df_bom_cleaned = df_bom[df_bom['ERP编号'].notna()]

# Create the subset where 'PLM编号' starts with '1.5.'
df_bom_electrical_panel = df_bom_cleaned[df_bom_cleaned['PLM编号'].str.startswith('1.5.')]

df_bom_electrical_panel = df_bom_electrical_panel[df_bom_electrical_panel['ERP编号'].str.match(r'^\d{8}$')]

df_bom_flipper = df_bom_cleaned[~df_bom_cleaned['PLM编号'].str.startswith('1.5.')]

df_bom_flipper = df_bom_flipper[df_bom_flipper['ERP编号'].str.match(r'^\d{8}$')]

df_bom_flipper = df_bom_flipper[df_bom_flipper['PLM编号'].str.startswith('1')]

# df_bom_flipper

df_finished_goods = df_bom_cleaned[df_bom_cleaned['物料种类'].str.match('产成品')]

# display(df_finished_goods)

# display(df_bom_flipper)

# Select 'ERP编号' and '所属装配体', and rearrange them
df_bom_basic = pd.DataFrame({
    'ERP编号': df_finished_goods['ERP编号'].astype(str) + '_1',
    '语言': '中文',  # All values in this column will be '中文'
    '所属装配体': df_finished_goods['所属装配体'],
    'BOM类型': '临时'  # All values in this column will be '临时'
})

display(df_bom_basic)

# Call the function to write the DataFrame to Excel
write_df_to_excel(df_bom_basic, 'BOM模板.xlsx', '物料清单基本信息', start_row=8, start_col=1)

/home/kira/projects/sap-byd-api/app


ModuleNotFoundError: No module named 'helper'